# On-demand InSAR Processing with ASF and ISCE2

## Search for and download InSAR data

In [1]:
import datetime
from pathlib import Path

import asf_search as asf
import numpy as np

In [2]:
bbox = [37, 38, -92, -91] # SNWE

In [3]:
working_dir = Path('/Users/jlmd9g/Documents/insar_testing/')
data_dir = working_dir / 'Data'

sd = '2020-01-01T00:00:00Z'
ed = '2020-02-01T23:59:59Z'

flight_dir = None # ASCENDING or DESCENDING

In [4]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'start': f'{sd}',
    'end': f'{ed}',
    'processingLevel': 'SLC',
    }

if flight_dir is not None:
    opts['flightDirection'] = flight_dir

In [5]:
def make_wkt(bbox):
    """Create a WKT from a simple SWNE bbox"""
    S, N, W, E = bbox
    return f'POLYGON(({W} {S}, {E} {S}, {E} {N}, {W} {N}, {W} {S}))'

def id_paths(res):
    """
    Print out the paths (InSAR tracks) from an ASF search.
    
    res should be the results from geo_search
    """
    paths = []
    for r in res:
        paths.append(r.properties['pathNumber'])
    paths.sort()
    return paths, list(set(paths))

def select_path(res):
    """
    Algorithm to quickly select a path to process. Take the path
    that has the highest number of acquisitions, and if multiple 
    with the same take the path that has the closest data in time 
    to the current

    res should be the results from geo_search
    """
    ref_time = datetime.datetime.utcnow() - datetime.timedelta(days=100*365)
    
    # Get the path IDs and paths
    paths, path_ids = id_paths(res)
    path_ids = np.array(path_ids)

    # Get the path ids that have the most acquisitions
    num_acq = []
    for p in path_ids:
        num_acq.append(np.sum(np.array(paths) == p))
    max_p = np.array(num_acq == np.max(num_acq))

    # if a tie, take the one that has the closest data
    # Initialize the search
    smallest_diff_time = datetime.datetime.utcnow() - ref_time
    end_dates = []
    path_to_use = path_ids[0]
    potential_paths = path_ids[max_p]
    if isinstance(potential_paths, int):
        potential_paths = [potential_paths]
    
    for r in res:
        if r.properties['pathNumber'] in potential_paths:
            end_time = r.properties['stopTime']
            diff_time = datetime.datetime.utcnow() - datetime.datetime.strptime(end_time, '%Y-%m-%dT%H:%M:%SZ')
            if diff_time < smallest_diff_time:
                smallest_diff_time = diff_time
                path_to_use = r.properties['pathNumber']

    return path_to_use

In [6]:
AOI = make_wkt(bbox)
print(AOI)

POLYGON((-92 37, -91 37, -91 38, -92 38, -92 37))


In [122]:
results = asf.geo_search(intersectsWith=AOI, **opts)
print(f'{len(results)} results found')

14 results found


In [123]:
paths, path_ids = id_paths(results)
print(f'Tracks {path_ids} were found')

Tracks [92, 165, 63] were found


In [124]:
picked_path = select_path(results)
print(f'Track {picked_path} was selected for processing')

Track 165 was selected for processing


In [128]:
opts['relativeOrbit'] = picked_path

In [129]:
opts

{'platform': 'SENTINEL-1',
 'start': '2020-01-01T00:00:00Z',
 'end': '2020-02-01T23:59:59Z',
 'processingLevel': 'SLC',
 'relativeOrbit': 165}

In [130]:
results = asf.geo_search(intersectsWith=AOI, **opts)
print(f'{len(results)} results found')

6 results found


## Downloading Data

In order to download data from ASF, you will need to have and set up your credentials. The simplest and recommended way to do this is to create a .netrc file, which should be placed in your home directory. This file will contain your credentials in the following format: 
```
machine notaserver.nasa.gov
    login batman@wayneenterprises.com
    password ironmansucks
```

In [141]:
results.download(path=data_dir)

In [144]:
!ls ~/Documents/insar_testing/Data

S1A_IW_SLC__1SDV_20200107T000354_20200107T000421_030687_03847E_0CDE.zip
S1A_IW_SLC__1SDV_20200107T000419_20200107T000446_030687_03847E_54C0.zip
S1A_IW_SLC__1SDV_20200119T000353_20200119T000420_030862_038AA0_046A.zip
S1A_IW_SLC__1SDV_20200119T000418_20200119T000445_030862_038AA0_7094.zip
S1A_IW_SLC__1SDV_20200131T000353_20200131T000420_031037_0390C2_9A20.zip
S1A_IW_SLC__1SDV_20200131T000418_20200131T000445_031037_0390C2_C6D2.zip


In [148]:
!/Users/jlmd9g/software/mambaforge/envs/insar/share/isce2/topsStack/stackSentinel.py --help

Using default ISCE Path: /Users/jlmd9g/software/mambaforge/envs/insar/lib/python3.11/site-packages/isce
Traceback (most recent call last):
  File "/Users/jlmd9g/software/mambaforge/envs/insar/share/isce2/topsStack/stackSentinel.py", line 17, in <module>
    from topsStack.Stack import config, run, sentinelSLC
ModuleNotFoundError: No module named 'topsStack'
